In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

warnings.filterwarnings("ignore")

import lamindb as ln
import pandas as pd
import spatialdata as sd

from spatialdata_db import SpatialDataDBCurator

→ connected lamindb: scverse/spatialdata-db


In [4]:
def replace_var_index(sdata):
    for table_name, table in sdata.tables.items():
        if isinstance(table.var, pd.DataFrame):
            table.var["gene_names"] = table.var.index
            table.var.set_index("gene_ids", inplace=True)
            sdata.tables[table_name] = table

# Upload Visium Spatial, Human large instestine

In [3]:
ln.track("NDcshmgyz9vh0000")

→ loaded Transform('NDcshmgy'), re-started Run('XNbeIVQt') at 2025-01-19 21:38:42 UTC
→ notebook imports: lamindb==0.77.2 pandas==2.2.3 spatialdata-db==0.0.1 spatialdata==0.2.7.dev14+gc07c363


In [6]:
DATA_PATH = "/lustre/groups/ml01/projects/2024_spatialdata_db/data"
uid = "1000j"

In [7]:
folder_path = None
for folder in os.listdir(DATA_PATH):
    if folder.startswith(str(uid)):
        folder_path = os.path.join(DATA_PATH, folder)
        break

if folder_path is None:
    print(f"Folder for UID {uid} not found.")

In [8]:
file = None
for file in os.listdir(folder_path):
    if file.endswith(".zarr"):
        break

if file is None:
    print(f"No zarr file found in folder for UID {uid}.")

file_path = os.path.join(folder_path, file)

In [10]:
sdata = sd.read_zarr(file_path)
replace_var_index(sdata)

In [11]:
sdc = SpatialDataDBCurator(sdata, organism="human")
sdc.validate()

• validating categoricals of 'sample' metadata...
• mapping "assay" on ExperimentalFactor.name
!   1 term is not validated: 'Visium'
    1 synonym found: "Visium" → "Visium Spatial Gene Expression"
    → curate synonyms via .standardize("assay")
• mapping "chemistry_version" on ULabel.name
!   1 term is not validated: 'v1'
    → fix typos, remove non-existent values, or save terms via .add_new_from("chemistry_version")
• mapping "organism" on Organism.name
!   1 term is not validated: 'Human'
    1 synonym found: "Human" → "human"
    → curate synonyms via .standardize("organism")
• mapping "tissue" on Tissue.name
!   1 term is not validated: 'large_intestine'
    → fix typos, remove non-existent values, or save terms via .add_new_from("tissue")
• mapping "disease" on Disease.name
!   1 term is not validated: 'large intestine colorectal cancer'
    → fix typos, remove non-existent values, or save terms via .add_new_from("disease")
✓ "license" is validated against ULabel.name
✓ "preproc

False

In [12]:
sdc.standardize("assay", accessor="sample")
sdc.add_new_from("chemistry_version", accessor="sample")
sdc.standardize("organism", accessor="sample")
sdc.add_new_from("tissue", accessor="sample")
sdc.add_new_from("disease", accessor="sample")

✓ standardized 1 synonym in "assay": "Visium" → "Visium Spatial Gene Expression"
✓ added 1 record with ULabel.name for "chemistry_version": 'v1'
✓ standardized 1 synonym in "organism": "Human" → "human"
✓ added 1 record with Tissue.name for "tissue": 'large_intestine'
✓ added 1 record with Disease.name for "disease": 'large intestine colorectal cancer'


In [13]:
sdc.validate()

• validating categoricals of 'sample' metadata...
✓ "assay" is validated against ExperimentalFactor.name
✓ "chemistry_version" is validated against ULabel.name
✓ "organism" is validated against Organism.name
✓ "tissue" is validated against Tissue.name
✓ "disease" is validated against Disease.name
✓ "license" is validated against ULabel.name
✓ "preproc_version" is validated against ULabel.name

• validating categoricals in table 'table'...
✓ "var_index" is validated against Gene.ensembl_gene_id



True

In [14]:
description = (
    "10X, "
    + sdata.attrs["sample"]["assay"]
    + ", "
    + sdata.attrs["sample"]["organism"]
    + ", "
    + sdata.attrs["sample"]["disease"]
    + ", "
    + sdata.attrs["sample"]["tissue"]
    + ", "
    + sdata.attrs["sample"]["date"][:4]
)
sdc.save_artifact(description=description)

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside /home/icb/lea.zimmermann/.cache/lamindb/65485167.zarr). Please see the documentation of 
         `is_self_contained()` to understand the implications of working with SpatialData objects that are not     
         self-contained.                                                                                           
INFO     The Zarr backing store has been changed from                                                              
         /lustre/groups/ml01/projects/2024_spatialdata_db/data/1000j__10X__Visium__Human__large_intestine__20220328
         __v1.3.0/1000j__10X__Visium__Human__large_intestine__20220328__v1.3.0_attrs.zarr the new file path:       
         /home/icb/lea.zimmermann/.cache/lamindb/65485167.zarr                                                     
... uploading 65485167.zarr: 100.0%
!    5 unique terms (100.00%) are no

Artifact(uid='8XBBsxnNHuWCOeOb0000', is_latest=True, description='10X, Visium, Human, large intestine colorectal cancer, large_intestine, 2022', suffix='.zarr', size=52401277, hash='TUrS6Myy91ExOsSws_zQ2A', n_objects=323, _hash_type='md5-d', _accessor='spatialdata', visibility=1, _key_is_virtual=True, storage_id=2, transform_id=63, run_id=237, created_by_id=8, created_at=2025-01-19 16:42:31 UTC)

In [4]:
ln.finish(ignore_non_consecutive=True)

→ finished Run('XNbeIVQt') after 0d 0h 0m 12s at 2025-01-19 21:38:55 UTC
→ go to: https://lamin.ai/scverse/spatialdata-db/transform/NDcshmgyz9vh0000
→ if you want to update your notebook without re-running it, use `lamin save /home/icb/lea.zimmermann/projects/spatialdata-db/spatialdata-db/scripts/upload_1000j_visium.ipynb`
